In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from superprimitive_fusion.scanner import capture_spherical_scans, render_rgb_view, virtual_mesh_scan
from superprimitive_fusion.utils import bake_uv_to_vertex_colours, polar2cartesian
from superprimitive_fusion.mesh_fusion_utils import get_mesh_components
from superprimitive_fusion.mesh_fusion import fuse_meshes

In [ ]:
mesh_gt = o3d.io.read_triangle_mesh("../data/posed-meshes/mustard-bottle/mustard-bottle.obj", enable_post_processing=True)

bake_uv_to_vertex_colours(mesh_gt)

mesh_gt.compute_vertex_normals()

bb = mesh_gt.get_minimal_oriented_bounding_box()
scale = np.mean(bb.get_max_bound())

In [ ]:
look_at = mesh_gt.get_center() + np.array([0, 0, 0.5])

In [ ]:
scans = capture_spherical_scans(
    meshlist=[mesh_gt],
    num_views=6,
    look_at=look_at,
    radius=1,
    width_px=360,
    height_px=240,
    fov=70,
    k=10,
    sampler="fibonacci",
)
meshes = [scan['mesh'][0][0] for scan in scans]

o3d.visualization.draw_geometries(meshes)

In [ ]:
meshes, _ = virtual_mesh_scan(
    meshlist=[mesh_gt],
    cam_centre=mesh_gt.get_center() + polar2cartesian(0.4, 60, 45),#45,150?
    look_at=mesh_gt.get_center(),
    k=10,
    max_normal_angle_deg=None,
    linear_depth_sigma=0,
    quadrt_depth_sigma=0
)
mesh1 = meshes[0][0]
weights1 = meshes[1][0]
o3d.visualization.draw_geometries([mesh1])

In [ ]:
meshes, _ = virtual_mesh_scan(
    meshlist=[mesh_gt],
    cam_centre=mesh_gt.get_center() + polar2cartesian(0.4, 60, 150),#45,150?
    look_at=mesh_gt.get_center(),
    k=10,
    max_normal_angle_deg=None,
    linear_depth_sigma=0,
    quadrt_depth_sigma=0
)
mesh2 = meshes[0][0]
weights2 = meshes[1][0]
o3d.visualization.draw_geometries([mesh2])

In [ ]:
m, w = fuse_meshes(
    mesh1=mesh1,
    mesh2=mesh2,
    weights1=weights1,
    weights2=weights2,
    fill_holes=False,
)

In [ ]:
o3d.visualization.draw_geometries([m])

In [ ]:
o3d.visualization.draw_geometries(
    meshes,
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

In [ ]:
j = 5
fused_mesh = meshes[0]
for i in range(1,j):
    fused_mesh = fuse_meshes(
        fused_mesh,
        meshes[i],
        h_alpha=3,
        nrm_shift_iters=2,
        shift_all=False,
        fill_holes=False,
    )

In [ ]:
o3d.visualization.draw_geometries(
    [fused_mesh],
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

In [ ]:
get_mesh_components(fused_mesh, show=False)